In [1]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModelForCausalLM

/root/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True).to("cuda:0")
# model.to("cuda:0")

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 11.97it/s]


In [3]:
# prepare image and text prompt, using the appropriate prompt template
# url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_v1_5_radar.jpg?raw=true"
# image = Image.open(requests.get(url, stream=True).raw)
image1 = Image.open("/root/TFVSN/test_images/test4.jpg")
image2 = Image.open("/root/TFVSN/test_images/test2.jpg")

# Define a chat histiry and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image") 
conversation = [
    {
      "role": "user",
      "content": [
          {"type": "text", "text": "describe the 2nd picture\n"},
          {"type": "image"},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
inputs = processor(images=[image1, image2], text=prompt, return_tensors="pt").to("cuda:0")
inputs
# inputs.pixel_values.shape
# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=1000)

print(processor.decode(output[0], skip_special_tokens=True))
# output

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Expanding inputs for image tokens in LLaVa-NeXT should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.


[INST]  
 
describe the 2nd picture
 [/INST] The image appears to be a stylized, colorful illustration of a bear standing in a forest clearing with a group of smaller animals around it. The bear is depicted with a friendly expression, and it's wearing a scarf. The smaller animals are various types, including what looks like a bird, a rabbit, and a squirrel, all of which are standing close to the bear.

The background features a gradient of colors, transitioning from a warm orange at the horizon to a cool blue at the top, suggesting either sunrise or sunset. The trees are depicted with a mix of green and blue hues, giving the impression of a serene, possibly magical forest.

The image is annotated with various labels and numbers, indicating some sort of data or performance metrics. These labels include "MMM-Vet," "MMM-Bench," "MMM-TextVQA," "MMM-SQA-IMG," "MMM-SQA-TextVQA," "MMM-SQA-TextVQA," "MMM-SQA-TextVQA," "MMM-SQA-TextVQA," "MMM-SQA-TextVQA," "MMM-SQA-TextVQA," "MMM-SQA-TextVQA," 